In [12]:
from metar import Metar
from datetime import datetime, timedelta
from metar import Metar
from util.utils import *
import pandas as pd
import re


In [13]:

def interpretaMetaR(metarString):

    if("METAF" in metarString):
        metarString = metarString.replace('METAF', 'METAR')
        metarString = metarString.replace('\n', '')


    station = None
    type = None
    hora_formated = None
    temperature = None
    dew_point = None
    windDirection = None
    windVelocity = None
    visibility = None
    pressure = None
    weather = None
    sky = None
    skyFeet= None

    try: 

        obs = Metar.Metar(metarString[:-1])

        for item in obs.string().split('\n'):
            if("station" in item):
                station = item.split(" ")[1]
            if("type" in item):
                type = " ".join(item.split(" ")[1:])
            if("time" in item):
                timeZ = " ".join(item.split(" ")[1:])
                data_obj = datetime.strptime(timeZ, "%a %b %d %H:%M:%S %Y")
                nova_data_obj = data_obj - timedelta(hours=3)
                hora_formated = nova_data_obj.strftime("%a %b %d %H:%M:%S %Y")
            if("temperature" in item):
                temperature = item.split(" ")[1]
            if("dew point" in item):
                dew_point = item.split(" ")[2]
            if("wind" in item):
                wind = (item.split(" ")[1:])
                windDirection = wind[0]
                windVelocity = wind[2]
            if("visibility" in item):
                visibility = item.split(" ")[1]
            if("pressure" in item):
                pressure = item.split(" ")[1]
            if("weather" in item):
                weather = " ".join(item.split(" ")[1:])
            if("sky" in item):
                skyString = " ".join(item.split(" ")[1:])
                if(' at ' in skyString):
                    sky = ''.join(skyString.split(' at ')[0])
                else:
                    sky = ''.join(skyString.split(',')[0])
                
                allFeets = re.findall(r"(\d+)",skyString)
                if len(allFeets) > 0:
                    skyFeet = allFeets[0]
    except:
        pass

    return station, type, hora_formated, temperature,dew_point,windDirection,windVelocity,visibility,pressure,weather,sky,skyFeet

def direction_to_degrees(df):
    direction_degrees = {
    'N': 0,
    'NNE': 22.5,
    'NE': 45,
    'ENE': 67.5,
    'E': 90,
    'ESE': 112.5,
    'SE': 135,
    'SSE': 157.5,
    'S': 180,
    'SSW': 202.5,
    'SW': 225,
    'WSW': 247.5,
    'W': 270,
    'WNW': 292.5,
    'NW': 315,
    'NNW': 337.5,
    'calm': 0,
    'variable': 0,
    }
    try:
        df['metar_windDirection'] = df['metar_windDirection'].map(direction_degrees)
    except:
        df['metaf_windDirection'] = df['metaf_windDirection'].map(direction_degrees)

    return df

def defineIndex(df):
    df = df.sort_values(by='hora_formatted')
    df.set_index('hora_formatted', inplace=True)
    df.index = pd.to_datetime(df.index, format='%A, %B %d, %Y %H:%M:%S')
    return df

def to_dummies(df):
    df_Dummies = pd.get_dummies(df['sky'])
    df_final = pd.concat([df, df_Dummies], axis=1)

    return df_final

def sort_df(df):
    df = df.drop_duplicates()
    df = df.dropna()
    df = df.sort_values(by=['aero','hora_formatted'])

    return df
def calculate_average(series):
    result = []
    for i in range(len(series) - 1):
        current_value = series.iloc[i]
        next_value = series.iloc[i + 1]
        average = (current_value + next_value) / 2
        result.append(average)
    return pd.Series(result, index=series.index[:-1])

def pipemediaTempoMetar(df, colunas):
    listSeries = []
    non_numeric_columns = []

    for coluna in colunas:
        if pd.api.types.is_numeric_dtype(df[coluna]):
            df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
            df_resampled_temp = df[coluna].resample('30T').mean()
            df_resampled_temp_filled = df_resampled_temp.fillna(method='ffill')
            tempSeries = calculate_average(df_resampled_temp_filled)
            listSeries.append(tempSeries)
        else:
            non_numeric_columns.append(coluna)

    dataframe_resultante = pd.concat(listSeries, axis=1)

    # Criar um novo DataFrame para armazenar as colunas não numéricas
    df_nao_numericas = df[non_numeric_columns]

    # Redefinir os índices dos DataFrames antes da concatenação
    dataframe_resultante.reset_index(drop=True, inplace=True)
    df_nao_numericas.reset_index(drop=True, inplace=True)

    # Concatenar o DataFrame resultante com o DataFrame das colunas não numéricas
    resultado_final = pd.concat([dataframe_resultante, df_nao_numericas], axis=1)

    return resultado_final

def read_metar(df,tabela):
    if (tabela == "metar"):
        resultado_df =  df['metar'].apply(interpretaMetaR)
        resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['metar_station', 'metar_type', 'metar_hora_formated', 'metar_station','metar_dew_point','metar_windDirection','metar_windVelocity','metar_visibility','metar_pressure','metar_weather','metar_sky','metar_skyFeet'] )
    else:
        resultado_df =  df['metaf'].apply(interpretaMetaR)
        resultado_df =  pd.DataFrame(resultado_df.tolist(),columns=['metaf_station', 'metaf_type', 'metaf_hora_formated', 'metaf_station','metaf_dew_point','metaf_windDirection','metaf_windVelocity','metaf_visibility','metaf_pressure','metaf_weather','metaf_sky','metaf_skyFeet'] )
    df_bronze = pd.concat([df, resultado_df], axis=1)
    df_bronze = direction_to_degrees(df_bronze)
    # df_bronze = sort_df(df_bronze)
    # df_bronze = df_bronze.reset_index(drop=True)
    # df_bronze['hora_formatted'] = pd.to_datetime(df_bronze['hora_formatted'])
    # df_bronze.set_index(['aero','hora_formatted'], inplace=True)
    return df_bronze
    df_media = pipemediaTempoMetar(df_bronze,['hora','aero','dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'])
    df_no_columns = df_bronze.drop(['dew_point', 'windDirection', 'windVelocity', 'visibility','pressure', 'weather', 'sky', 'skyFeet'],axis=1)
    return df_media,df_bronze



In [14]:
df = pd.read_csv("../data/processed_data/merged_bronze.csv")

In [15]:
df = read_metar(df,"metar")
df = read_metar(df,"metaf")

In [18]:
df.head()

,Unnamed: 0,flightid,origem,destino,dt_dep,dt_arr,dt_dep_formatted,dt_arr_formatted,hora_real,nova_cabeceira,...,metaf_hora_formated,metaf_station,metaf_dew_point,metaf_windDirection,metaf_windVelocity,metaf_visibility,metaf_pressure,metaf_weather,metaf_sky,metaf_skyFeet
0,8025,b6fae10418d00900c1a8e8388af71abf,SBGR,SBBR,1679325532000,1679330517000,"Monday, March 20, 2023 12:18:52","Monday, March 20, 2023 13:41:57",1679326233000,28L,...,Wed Sep 20 14:00:00 2023,26.0,15.0,ENE,8,10000,1013.0,None,None,None
1,8364,e27318884f3d6d589f8575c018d5b589,SBGR,SBBR,1679394689000,1679399257000,"Tuesday, March 21, 2023 07:31:29","Tuesday, March 21, 2023 08:47:37",1679395529000,10R,...,Thu Sep 21 09:00:00 2023,22.0,14.0,E,7,greater,1018.0,None,a few clouds,2800
2,8674,fbc4b661c16c196766b6bb4726b07fe2,SBGR,SBBR,1679357934000,1679362375000,"Monday, March 20, 2023 21:18:54","Monday, March 20, 2023 22:32:55",1679358906000,10L,...,Wed Sep 20 23:00:00 2023,20.0,17.0,NNW,2,7000,1016.0,None,broken clouds,2700
3,9006,05fa8ded3b230cf1997cb6eb4f7e271f,SBGR,SBBR,1679403588000,1679408295000,"Tuesday, March 21, 2023 09:59:48","Tuesday, March 21, 2023 11:18:15",1679404868000,10R,...,Thu Sep 21 11:00:00 2023,24.0,15.0,E,8,greater,1018.0,None,scattered clouds,3300
4,9362,49c9529ddfeb567781fe31ea2d0dc9b0,SBGR,SBBR,1679531627000,1679536433000,"Wednesday, March 22, 2023 21:33:47","Wednesday, March 22, 2023 22:53:53",1679533175000,10R,...,Fri Sep 22 23:00:00 2023,21.0,16.0,ENE,2,10000,1016.0,None,None,None


In [ ]:
df.to_csv("../data/processed_data/merged_plate.csv")